In [0]:
!git clone https://github.com/Qidian213/deep_sort_yolov3.git

Cloning into 'deep_sort_yolov3'...
remote: Enumerating objects: 195, done.
remote: Total 195 (delta 0), reused 0 (delta 0), pack-reused 195
Receiving objects: 100% (195/195), 10.72 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [3]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
! pip install imutils

  Using cached imutils-0.5.3.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25856 sha256=87b75505060f3fe3b218b614408c2c581c83be56ac097675a6f3a42ec8199a2d
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\27\4d\3a\6dcdf7c3ebc87bf1ae013d96c9cf060ccfe334bb5ee769f377
Successfully built imutils


In [4]:
! pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached scipy-1.5.2-cp36-cp36m-win_amd64.whl (31.2 MB)
  Using cached h5py-2.10.0-cp36-cp36m-win_amd64.whl (2.4 MB)


In [3]:
! pip install tensorflow --user

  Using cached tensorflow-2.3.0-cp36-cp36m-win_amd64.whl (342.5 MB)
Processing c:\users\lenovo\appdata\local\pip\cache\wheels\93\2a\eb\e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc\termcolor-1.1.0-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.31.0-cp36-cp36m-win_amd64.whl (2.6 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Processing c:\users\lenovo\appdata\local\pip\cache\wheels\c3\af\84\3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679\absl_py-0.9.0-py3-none-any.whl
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-3.12.4-cp36-cp36m-win_amd64.whl (1.1 MB)
  Using cached scipy-1.4.1-cp36-cp36m-win_amd64.whl (30.8 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Processing c:\users\lenovo\appdata\local\pip\cache\wheels\32\42\7f\23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63\wrapt-1.12.1-cp36-cp36m-win_amd64.whl
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-1.20.1-py2.py3-none-any.whl (91 kB)
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10 kB)
  Using cached chardet-3.

In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import division, print_function, absolute_import
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time
import os
import math
from timeit import time
import warnings
import sys
import cv2
import numpy as np
from PIL import Image
from yolo import YOLO

from deep_sort import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet
from deep_sort.detection import Detection as ddet
warnings.filterwarnings('ignore')

def main(yolo):

   # Definition of the parameters
    max_cosine_distance = 0.3
    nn_budget = None
    nms_max_overlap = 1.0
    
   # deep_sort 
    model_filename = 'model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename,batch_size=1)
    
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    writeVideo_flag = True 
    
    video_capture = cv2.VideoCapture('soccer_clip.mp4')
    pts6 = deque(maxlen=1000)
    ptsp6 = deque(maxlen = 1000)
    dist = 0

    if writeVideo_flag:
    # Define the codec and create VideoWriter object
        w = int(video_capture.get(3))
        h = int(video_capture.get(4))
        fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter('next_9_final_outputspan_15.avi', fourcc, fps, (w, h))
        list_file = open('detection.txt', 'w')
        frame_index = -1 
        
    fps = 0.0
    cou = 0
    speed = 0
    while True:
        ret, frame = video_capture.read()  # frame shape 640*480*3
        if ret != True:
            break
        t1 = time.time()

       # image = Image.fromarray(frame)
        image = Image.fromarray(frame[...,::-1]) #bgr to rgb
        boxs = yolo.detect_image(image)
       # print("box_num",len(boxs))
        features = encoder(frame,boxs)
        
        # score to 1.0 here).
        detections = [Detection(bbox, 1.0, feature) for bbox, feature in zip(boxs, features)]
        
        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]
        
        # Call the tracker
        tracker.predict()
        tracker.update(detections)
        countp = 0
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue 
            bbox = track.to_tlbr()
            # i1 = bbox[0]+ 0.35*(bbox[2]-bbox[0])
            # i2 = bbox[1] + 0.04*(bbox[3]-bbox[1])
            # i3 = bbox[2] - 0.35*(bbox[2]-bbox[0])
            # i4 = bbox[1] + 0.2*(bbox[3]-bbox[1])

            # cv2.rectangle(frame, (int(i1), int(i2)), (int(i3), int(i4)),(255,255,255), 2)
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
           
            centre = (int((bbox[0]+bbox[2])/2), int(bbox[3]))
            textr = (int(bbox[0]), int(bbox[1]))
            countp = countp+1
            if track.track_id ==6:
                pts6.appendleft(centre)
                print(centre)
                ptsp6.appendleft(textr)
        for det in detections:
            bbox = det.to_tlbr()
            cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,0,0), 2)
        pts = 'pts'   
        #cv2.imshow('', frame)
        cou = cou +1
         
        for i in range(1, len(pts6)):
		         if pts6[i - 1] is None or pts6[i] is None:
		                	    continue
                   
                  #  elif i = len(pts6) - 1:
		              #         dist = dist + math.hypot(pts6[i][0] - pts6[i-1][0], pts6[i][1] - pts6[i-1][1])
                  #         cv2.putText(frame, str(dist),(int(ptsp6[i][0]), int(ptsp[i][1])),0, 5e-3 * 200, (0,255,0),2)
		         cv2.line(frame, pts6[i - 1], pts6[i], (255, 0, 0), 3)
        if cou >3:
            
                    dist = dist + math.hypot(pts6[0][0] - pts6[1][0], pts6[0][1] - pts6[1][1])/40
                    cv2.putText(frame, 'distance_travelled (in metre): '+str(round(dist,3)),(int(ptsp6[0][0]), int(ptsp6[0][1])),0, 5e-3 * 200, (0,255,0),2)
                    cv2.putText(frame, 'Speed (metre/second): '+str(round(speed,3)),(int(ptsp6[0][0] +8), int(ptsp6[0][1] + 28)),0, 5e-3 * 200, (0,255,0),2)
                    if cou%5 ==0 and cou >8:
                        speed = 3.2*math.hypot(pts6[0][0] - pts6[5][0], pts6[0][1] - pts6[5][1])/40
                        
                        cv2.putText(frame, 'Speed (metre/second): '+str(round(speed,3)),(int(ptsp6[0][0] +8), int(ptsp6[0][1] + 18)),0, 5e-3 * 200, (0,255,0),2)
                        cv2.putText(frame, 'Players Detected: '+str(countp),(10,30),0, 5e-3 *180, (0,0,255),2)
        if writeVideo_flag:
            # save a frame
            out.write(frame)
            frame_index = frame_index + 1
            list_file.write(str(frame_index)+' ')
            if len(boxs) != 0:
                for i in range(0,len(boxs)):
                    list_file.write(str(boxs[i][0]) + ' '+str(boxs[i][1]) + ' '+str(boxs[i][2]) + ' '+str(boxs[i][3]) + ' ')
            list_file.write('\n')
            
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        print("fps= %f"%(fps))
        
        # Press Q to stop!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    if writeVideo_flag:
        out.release()
        list_file.close()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main(YOLO())


Using TensorFlow backend.
C:\Users\Lenovo\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lenovo\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lenovo\Anaconda3\lib\site-packages\tensorboard\

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




fps= 0.146933
fps= 0.614883
(978, 707)
fps= 0.876496
(969, 707)
fps= 1.013199
(972, 707)
fps= 1.073083
(979, 707)
fps= 1.058767
(977, 706)
fps= 1.070785
fps= 1.082704
fps= 1.085102


IndexError: deque index out of range

In [0]:
%cd PlayerDetection/

[Errno 2] No such file or directory: 'PlayerDetection/'
/content/drive/My Drive/code/deep_sort_yolov3/PlayerDetection


In [0]:
%cd '/content/drive/My Drive/code/deep_sort_yolov3/PlayerDetection'

/content/drive/My Drive/code/deep_sort_yolov3/PlayerDetection


In [0]:
!python player_detection.py

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame